# 使用NTCIR給的relation資料集

In [2]:
!pip install transformers datasets accelerate


[notice] A new release of pip is available: 23.0 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!nvidia-smi

Wed Apr 26 23:07:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.48       Driver Version: 517.48       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   35C    P8    21W / 200W |   1681MiB /  8192MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 抓dataset

In [4]:
from datasets import list_datasets, load_dataset
from pprint import pprint

c:\Users\lin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

import json
with open("train_relation.json","r")as f:
    pass
    dataset=json.load(f)
    print(type(dataset))

<class 'list'>


## dataset裡分成三個 train(訓練用) validation(每次訓縣完測試訓練情形用) 其中把train的520筆切出來當成test切出來當成test

In [6]:
print(dataset[5519])
print(dataset[0])
print(dataset[8])

['We continue to see and saw again installed base growth across all of our customer segments from the enterprise to small business.', 'And I think we were quite encouraged by E5 performance, particularly in Q4 as we ended the quarter, which was great to see.', 0]
['Japan as a geography for us is a high transactional market.', 'The improvement in that in Q3 is obviously very high margin and also the bottom.', 1]
["I think there's a tendency in this industry to call everything new the next computer platform.", 'However, that said, I think AR can be huge.', 2]


In [7]:
train_data=dataset[0:5000]
test_data=dataset[5000:5521]
with open("dev_relation.json","r")as f2:
    eval_data=json.load(f2)
#eval_data=dataset['validation']

In [8]:
import json
from pprint import pprint

pprint(dataset[5519])
pprint(dataset[0])
pprint(dataset[8])

['We continue to see and saw again installed base growth across all of our '
 'customer segments from the enterprise to small business.',
 'And I think we were quite encouraged by E5 performance, particularly in Q4 '
 'as we ended the quarter, which was great to see.',
 0]
['Japan as a geography for us is a high transactional market.',
 'The improvement in that in Q3 is obviously very high margin and also the '
 'bottom.',
 1]
["I think there's a tendency in this industry to call everything new the next "
 'computer platform.',
 'However, that said, I think AR can be huge.',
 2]


## 把每一筆資料集的每一項拆開

In [9]:
def read_data(dataset):
    #open file
    sentence1=[]
    sentence2=[]
    label=[]
    #idx=[]
    for text in dataset:
        sentence1.append(text[0])
        sentence2.append(text[1])
        label.append(text[2])
    return sentence1,sentence2,label

In [10]:
"""def read_data(dataset):
    #open file
    sentence1=[]
    sentence2=[]
    label=[]
    #idx=[]
    for text in dataset:
        sentence1.append(text[0]+" [SEP] "+text[1])
        sentence2.append(text[1])
        label.append(text[2])
    return sentence1,sentence2,label"""

'def read_data(dataset):\n    #open file\n    sentence1=[]\n    sentence2=[]\n    label=[]\n    #idx=[]\n    for text in dataset:\n        sentence1.append(text[0]+" [SEP] "+text[1])\n        sentence2.append(text[1])\n        label.append(text[2])\n    return sentence1,sentence2,label'

In [11]:
train_sen1,train_sen2,trainlabel=read_data(train_data)
#trainlabel=train_data['label']
print(len(train_sen1))
print(train_sen1[0])

5000
Japan as a geography for us is a high transactional market.


In [12]:
test_sen1,test_sen2,testlabel=read_data(test_data)
print(len(test_sen1))
print(test_sen1[0])

521
Not only is .NET first class but Linux is first class, Java is first class.


In [13]:
eval_sen1,eval_sen2,evallabel=read_data(eval_data)
print(len(eval_sen1))
print(eval_sen1[0])

690
It helps us identify content that might be offensive or graphic that might violate the policies of Facebook so we can flag that and review that better.


In [14]:
print('train 資料集數量= ',len(train_data))
print('eval 資料集數量= ',len(eval_data))
print('test 資料集數量= ',len(test_data))

train 資料集數量=  5000
eval 資料集數量=  690
test 資料集數量=  521


In [15]:
print(trainlabel)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 

# 選擇和使用tokenizer

In [16]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
from transformers import DebertaTokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

#from transformers import DebertaTokenizerFast

#tokenizer = DebertaTokenizerFast.from_pretrained("microsoft/deberta-base")

In [18]:
tokenizer("Hello world")["input_ids"]

[1, 31414, 232, 2]

In [19]:
tokenizer(" Hello world","hello world")["input_ids"]

[1, 20920, 232, 2, 42891, 232, 2]

In [20]:
tokenizer

PreTrainedTokenizerFast(name_or_path='microsoft/deberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [21]:
train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True, return_tensors="pt")
print(train_encodings)
eval_encodings = tokenizer(eval_sen1, eval_sen2, truncation=True, padding=True, return_tensors="pt")

{'input_ids': tensor([[    1, 21318,    25,  ...,     0,     0,     0],
        [    1,  2847,     6,  ...,     0,     0,     0],
        [    1, 13437,    14,  ...,     0,     0,     0],
        ...,
        [    1,   133,    78,  ...,     0,     0,     0],
        [    1,   100,    74,  ...,     0,     0,     0],
        [    1,   170,   348,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [22]:
#把第二句改成label試試
#train_encodings = tokenizer(train_sen1, truncation=True, padding=True)
#print(train_encodings)
#eval_encodings = tokenizer(eval_sen1, truncation=True, padding=True)

In [23]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [24]:
print(tokenizer.decode(train_encodings['input_ids'][0]))
print(train_encodings['input_ids'][0])
print("token_type_ids\n", train_encodings['token_type_ids'][0])
print("attention_mask\n", train_encodings['attention_mask'][0])

print(len(train_encodings['input_ids'][0]))
print(len(train_encodings['token_type_ids'][0]))

[CLS]Japan as a geography for us is a high transactional market.[SEP]The improvement in that in Q3 is obviously very high margin and also the bottom.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][

## 加入label(答案) deberta 沒有label?

In [25]:
def add_targets(encodings,label):
    encodings.update({'labels':label})
add_targets(train_encodings,trainlabel)
add_targets(eval_encodings,evallabel)


In [26]:
print(train_encodings.keys())
print(tokenizer.decode(train_encodings['labels']))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
[CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][SEP][SEP][CLS][CLS][CLS][PAD][PAD][CLS][CLS][PAD][PAD][PAD][CLS][CLS][CLS][CLS][PAD][PAD][CLS][CLS][PAD][PAD][PAD][CLS][CLS][CLS][PAD][PAD][PAD][PAD][CLS][CLS][SEP][CLS][CLS][PAD][CLS][CLS][CLS][CLS][CLS][CLS][PAD][PAD][PAD][PAD][CLS][PAD][CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][PAD][PAD][PAD][CLS][CLS][PAD][PAD][CLS][CLS][CLS][CLS][PAD][CLS][PAD][PAD][PAD][CLS][CLS][CLS][CLS][CLS][CLS][PAD][PAD][PAD][PAD][PAD][CLS][PAD][PAD][PAD][PAD][CLS][PAD][PAD][CLS][CLS][CLS][PAD][CLS][CLS][CLS][CLS][CLS][PAD][CLS][PAD][CLS][CLS][PAD][CLS][PAD][PAD][SEP][CLS][PAD][PAD][CLS][PAD][CLS][CLS][PAD][CLS][CLS][CLS][PAD][PAD][PAD][CLS][PAD][PAD][CLS][CLS][CLS][CLS][CLS][CLS][CLS][CLS][PAD][PAD][PAD][CLS][PAD][CLS][CLS][CLS][CLS][PAD][CLS][CLS][CLS][PAD][PAD][PAD][CLS][CLS][CLS][PAD][CLS][CLS][CLS][CLS][CLS][CLS][CLS][PAD][PAD][PAD][CLS][CLS][PAD][CLS][CLS][CLS][PAD][PAD

In [27]:
len(eval_encodings['input_ids'])

690

# 定義dataset 並轉換成tensor格式


In [28]:
#eval[idx].clone().detach()
#torch.tensor(eval[idx])
from torch.utils import data
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [29]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [30]:
train_dataset[2]

C:\Users\lin\AppData\Local\Temp\ipykernel_22488\565021431.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}


{'input_ids': tensor([    1, 13437,    14,    18,  1088,   149,     5,   276,   622,  2329,
         33285,     6,    53,    52,   214,  1782,   269,   205,  5883,     8,
           205,   434,    89,     4,     2,  1121,  1110,     9,     5,   709,
            15,     5,  4579,   526,     6,    52,   214,    45,  4010,  3433,
            66,  1838,   903,     6,    25,    47,   216,     6,     2,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [31]:
len(eval_dataset)

690

# 載入模型架構

In [32]:
"""import torch
from transformers import AutoTokenizer, DebertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)

model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=num_labels)

labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss"""

'import torch\nfrom transformers import AutoTokenizer, DebertaForSequenceClassification\n\ntokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")\nmodel = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base")\n\ninputs = tokenizer("Hello, my dog is cute", return_tensors="pt")\n\nwith torch.no_grad():\n    logits = model(**inputs).logits\n\npredicted_class_id = logits.argmax().item()\n\n# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`\nnum_labels = len(model.config.id2label)\n\nmodel = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=num_labels)\n\nlabels = torch.tensor([1])\nloss = model(**inputs, labels=labels).loss'

In [33]:
#print(model.config.id2label)
#print(type(model.config.id2label))

In [34]:
num_label={0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

In [35]:
from transformers import AutoTokenizer, DebertaForSequenceClassification
from transformers import AutoModel,DebertaModel
from transformers import AutoConfig

config = AutoConfig.from_pretrained("microsoft/deberta-base")
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=3) 
#model = DebertaModel.from_pretrained("microsoft/deberta-base")

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier

In [36]:
print(model)

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

## 該來訓練模型囉

In [37]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 4     # 設定 training batch size 
eval_batch_size = 1      # 設定 eval batch size
num_train_epochs = 70      # 設定 epoch 

In [38]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [39]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 87500


c:\Users\lin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [40]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

#eval_dataloader
metric = load_metric("accuracy")

C:\Users\lin\AppData\Local\Temp\ipykernel_22488\4075806145.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


真正開始訓練

In [41]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = './content/drive/Shareddrives'  # your folder


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):#trange是print進度條的方式
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    #print(batch)
    outputs = model(**batch)
    #loss = outputs
    loss = outputs.loss
    loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break
      
  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    #pred=outputs['labels']
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  logger.info(f"epoch {epoch}: {eval_metric}")
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(epoch), save_function=accelerator.save)

04/26/2023 23:07:40 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no

04/26/2023 23:07:40 - INFO - __main__ - ***** Running training *****
04/26/2023 23:07:40 - INFO - __main__ -   Num examples = 5000
04/26/2023 23:07:40 - INFO - __main__ -   Num Epochs = 70
04/26/2023 23:07:40 - INFO - __main__ -   Instantaneous batch size per device = 4
04/26/2023 23:07:40 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 4
04/26/2023 23:07:40 - INFO - __main__ -   Gradient Accumulation steps = 1
04/26/2023 23:07:40 - INFO - __main__ -   Total optimization steps = 87500
Epoch:   0%|          | 0/70 [00:00<?, ?it/s]C:\Users\lin\AppData\Local\Temp\ipykernel_22488\565021431.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

{'epoch': 0, 'step': 0, 'loss': 1.123324990272522}


{'epoch': 0, 'step': 50, 'loss': 0.7944974303245544}


{'epoch': 0, 'step': 100, 'loss': 0.20277699828147888}


{'epoch': 0, 'step': 150, 'loss': 0.701838493347168}


...

{'epoch': 69, 'step': 1050, 'loss': 0.3689974844455719}


{'epoch': 69, 'step': 1100, 'loss': 0.3738884925842285}


{'epoch': 69, 'step': 1150, 'loss': 0.5667203664779663}


{'epoch': 69, 'step': 1200, 'loss': 0.5654047727584839}


Iteration: 100%|█████████▉| 1249/1250 [03:03<00:00,  6.80it/s]
04/27/2023 03:05:34 - INFO - __main__ - ***** Running eval *****
eval Iteration: 100%|██████████| 690/690 [00:18<00:00, 36.55it/s]
04/27/2023 03:05:52 - INFO - __main__ - epoch 69: {'accuracy': 0.6985507246376812}
Epoch: 100%|██████████| 70/70 [3:58:13<00:00, 204.19s/it]


## 最好的某次訓練成果

In [55]:
print(best_epoch)

{'epoch': 0, 'acc': 0.6985507246376812}


# 訓練成果驗證

In [43]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
from transformers import AutoTokenizer, DebertaForSequenceClassification, AutoConfig

In [44]:
#cd drive

In [45]:
#config = BertConfig.from_pretrained("./drive/Shareddrives/epoch_0/config.json") 
#model = BertForSequenceClassification.from_pretrained("./drive/Shareddrives/epoch_0/pytorch_model.bin", config = config).to(device)

In [46]:
config = AutoConfig.from_pretrained("./content/drive/Shareddrives/epoch_0/config.json")
model = DebertaForSequenceClassification.from_pretrained("./content/drive/Shareddrives/epoch_0/pytorch_model.bin", config = config).to(device) 

In [60]:
def mrpc_model(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

### 可以拿來玩的地方

In [56]:
sen1="And obviously that all translates into the revenue growth as well across all of the marketing solutions, talent solutions, and sales solutions"
sen2="First of all, it is very important for us to ensure that we did everything to enable LinkedIn to keep their product and cultural ethos of putting members first and innovating on their behalf."
#sen1="lisa is a singer"
#sen2="lisa is not a singer"

predict = mrpc_model(model, sen1, sen2)
print("sentence= : ", sen1)
print("sentence= : ", sen2)

print("predict_label : ", predict.item())
if predict.item()==0:
  print("none")
elif predict.item()==1:
  print('support')
else:
  print("attack")


sentence= :  And obviously that all translates into the revenue growth as well across all of the marketing solutions, talent solutions, and sales solutions
sentence= :  First of all, it is very important for us to ensure that we did everything to enable LinkedIn to keep their product and cultural ethos of putting members first and innovating on their behalf.
predict_label :  1
support


In [61]:
cnt=0
errorcnt=0
ss=0
sn=0
sa=0
ns=0
nn=0
na=0
a_s=0
an=0
aa=0

for i in range(len(test_data)):
  cnt+=1
  sen1=test_data[i][0]
  sen2=test_data[i][1]
  predict=mrpc_model(model,sen1,sen2)
  if predict.item()!=test_data[i][2]:
    errorcnt+=1
  if predict.item()==0 and test_data[i][2]==0:
    nn+=1
  if predict.item()==0 and test_data[i][2]==1:
    ns+=1
  if predict.item()==0 and test_data[i][2]==2:
    na+=1
  if predict.item()==1 and test_data[i][2]==0:
    sn+=1
  if predict.item()==1 and test_data[i][2]==1:
    ss+=1
  if predict.item()==1 and test_data[i][2]==2:
    sa+=1
  if predict.item()==2 and test_data[i][2]==0:
    an+=1
  if predict.item()==2 and test_data[i][2]==1:
    a_s+=1
  if predict.item()==2 and test_data[i][2]==2:
    aa+=1
print(cnt)
print(errorcnt)
print("none and none = ",nn)
print("none and support = ",ns)
print("none and attack = ",na)
print("support and none = ",sn)
print("support and support = ",ss)
print("support and attack = ",sa)
print("attack and none = ",an)
print("attack and support = ",a_s)
print("attacl and attack = ",aa)


521
159
none and none =  0
none and support =  0
none and attack =  0
support and none =  152
support and support =  362
support and attack =  7
attack and none =  0
attack and support =  0
attacl and attack =  0


In [62]:
n_precision=(nn/(nn+ns+na+1))
n_recall=(nn/(nn+sn+an+1))
s_precision=(ss/(sn+ss+sa+1))
s_recall=(ss/(ns+ss+a_s+1))
a_precision=(aa/(an+a_s+aa+1))
a_recall=(aa/(na+sa+aa+1))

n_f1=(2*n_precision*n_recall)/(n_precision+n_recall+1)
s_f1=(2*s_precision*s_recall)/(s_precision+s_recall+1)
a_f1=(2*a_precision*a_recall)/(a_precision+a_recall+1)

print("acc = " ,1-(errorcnt/cnt))
print("none presion = ",(nn/(nn+ns+na+1)))
print("none recall = ",(nn/(nn+sn+an+1)))
print("support presion = ",(ss/(sn+ss+sa+1)))
print("support recall = ",(ss/(ns+ss+a_s+1)))
print("attack presion = ",(aa/(an+a_s+aa+1)))
print("attack recall = ",(aa/(na+sa+aa+1)))
print("none f1 score = ",n_f1)
print("support f1 score = ",s_f1)
print("attack f1 score = ",a_f1)

acc =  0.6948176583493282
none presion =  0.0
none recall =  0.0
support presion =  0.6934865900383141
support recall =  0.9972451790633609
attack presion =  0.0
attack recall =  0.0
none f1 score =  0.0
support f1 score =  0.5140431808196824
attack f1 score =  0.0


結果存進google drive

In [51]:
"""from google.colab import drive
drive.mount('/content/gdrive')"""

"from google.colab import drive\ndrive.mount('/content/gdrive')"

In [52]:
#cd gdrive/MyDrive/Colab Notebooks

In [53]:
#torch.save(model,"./test_model2")

文全給我的東西

In [54]:
import torch
from torch import nn
from transformers import BertConfig,BertModel

class TaipeiClassifier(nn.Module):
    def __init__(self, n_classes):
        super(TaipeiClassifier, self).__init__()
        self.config = BertConfig.from_pretrained('bert-base-chinese')
        self.bert = BertModel.from_pretrained('bert-base-chinese', config=self.config)
        self.n_classes = n_classes
        self.drop = nn.Dropout(p=0.1, inplace=False)
        self.fc = nn.Linear(self.config.hidden_size, self.n_classes)

    def forward(self, input_ids, token_type_ids, attention_mask, labels = None):
        bert_out = self.bert(input_ids = input_ids, 
              token_type_ids = token_type_ids, 
              attention_mask = attention_mask,
              output_hidden_states=True)
        bert_out = bert_out[1] #get vector [cls]
        bert_out = self.drop(bert_out)
        predicted = self.fc(bert_out)

        predicted_value, predicted_class = torch.max(predicted, 1)

        if labels is not None:
            criterion = nn.CrossEntropyLoss()
            loss = criterion(predicted, labels)
            return predicted_class, loss
        else:
            return predicted_class